## Trying Music21

In [11]:
from music21 import converter, instrument, note, chord
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
Path_to_midi = 'RTML Project Data'
notes = []
for file in glob.glob(Path_to_midi+'/*/*.mid'):
    midi = converter.parse(file)
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: # file has instrument parts
        print("parts")
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        print("Flat")
        notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts
parts


In [7]:
n_vocab=len(set(notes))

In [8]:
n_vocab

82

In [9]:
import numpy as np
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
n_patterns = len(network_input)
    
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)

# network_output=np.eye(n_vocab, dtype='uint8')[network_output]

# LSTM

In [150]:
class LSTM(nn.Module):

    def __init__(self, hidden_size, sequence_length, n_vocab):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm=nn.LSTM(1,hidden_size,dropout = 0.3,batch_first=False).float()
        self.lstm2=nn.LSTM(hidden_size,hidden_size, dropout = 0.3,batch_first=False).float()
        self.lstm3=nn.LSTM(hidden_size,hidden_size).float()
        self.batch_normalization = nn.BatchNorm1d(num_features=hidden_size)
        self.batch_normalization2 = nn.BatchNorm1d(num_features=256)
        self.drop_out = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_size,256)
        self.out_layer = nn.Linear(256,n_vocab)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
        
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2output = nn.Linear(hidden_size, n_vocab)
        
    def init_hidden(self,sequence_length,hidden_size):
        return (torch.randn(1,sequence_length, hidden_size), torch.randn(1,sequence_length, hidden_size),
                torch.randn(1,sequence_length, hidden_size), torch.randn(1,sequence_length, hidden_size),
                torch.randn(1,sequence_length, hidden_size), torch.randn(1,sequence_length, hidden_size))

    def forward(self, input_sequence,hidden):
#         print(hidden[0].shape)
#         print(hidden[1].type())
#         print(input_sequence.shape)
        output,(h1,c1) = self.lstm(input_sequence,(hidden[0],hidden[1]))
        output,(h2,c2) = self.lstm2(output,(hidden[2],hidden[3]))
        output,(h3,c3) = self.lstm3(output,(hidden[4],hidden[5]))
#         print(output.shape)
        output[0][:-1].detach()
        output = output.squeeze()[-1]
#         output.requires_grad = True
#         output = self.batch_normalization(output.squeeze())
        output = self.drop_out(output)
        output = self.linear(h3[0][-1])
        output = self.relu(output)
#         output = self.batch_normalization2(output.squeeze())
        output = self.drop_out(output)
        output = self.out_layer(output)
#         print(output.shape)
        output = self.softmax(output)
        output = output.unsqueeze(0)
        return output

In [152]:
lstm_model = LSTM(512,100,n_vocab)

In [153]:
n_i = torch.from_numpy(network_input).float()

In [155]:
h_i = lstm_model.init_hidden(100,512)

In [156]:
lstm_model(n_i[0].unsqueeze(0),h_i).argmax()

tensor(345)

In [ ]:
criteron = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(lstm_model.parameters(),lr=0.01)
lstm_model.train()
for i in range(2,20):
#     lstm_model.zero_grad()
    a=lstm_model(input_sequence = n_i[i].unsqueeze(0),hidden=h_i)
    print("Predicted output is: ",a.argmax().item())
    print("Actual output is: ",network_output[i])
    loss = criteron(a,torch.tensor([network_output[i]]))
    print("Loss is: ",loss.item())
#     optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# LSTM CELL

In [12]:
class LSTMC(nn.Module):

    def __init__(self, hidden_size, sequence_length, n_vocab):
        super(LSTMC, self).__init__()
        self.hidden_size = hidden_size
        self.lstm=nn.LSTMCell(1,hidden_size).float()
        self.lstm2=nn.LSTMCell(hidden_size,hidden_size).float()
        self.lstm3=nn.LSTMCell(hidden_size,hidden_size).float()
        self.batch_normalization = nn.BatchNorm1d(num_features=hidden_size)
        self.batch_normalization2 = nn.BatchNorm1d(num_features=256)
        self.drop_out = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_size,256)
        self.out_layer = nn.Linear(256,n_vocab)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
        
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        
    def init_hidden(self,sequence_length,hidden_size,batch_size):
        return (torch.randn(batch_size, hidden_size,device=device), torch.randn(batch_size, hidden_size,device=device),
                torch.randn(batch_size, hidden_size,device=device), torch.randn(batch_size, hidden_size,device=device),
                torch.randn(batch_size, hidden_size,device=device), torch.randn(batch_size, hidden_size,device=device))

    def forward(self, input_sequence,hidden):
#         print(hidden[0].shape)
#         print(hidden[1].type())
#         print(input_sequence.shape)
        h1,c1 = self.lstm(input_sequence,(hidden[0],hidden[1]))
        h1 = self.drop_out(h1)
        h2,c2 = self.lstm2(h1,(hidden[2],hidden[3]))
        h2 = self.drop_out(h2)
        h3,c3 = self.lstm3(h2,(hidden[4],hidden[5]))
#         print(h3.shape)
        if(h1.size(0)>1):
            output = self.batch_normalization(h3)
        else:
            output = h3
        output = self.drop_out(output)
        output = self.linear(output)
        output = self.relu(output)
        if(output.size(0)>1):
            output = self.batch_normalization2(output)
        output = self.drop_out(output)
        output = self.out_layer(output)
        output = self.softmax(output)
        return output

In [13]:
device = 'cuda:1'
n_i = torch.tensor(network_input,device=device).float()

In [14]:
lstm_cell_model = LSTMC(512,100,n_vocab)
lstm_cell_model.to(device)

LSTMC(
  (lstm): LSTMCell(1, 512)
  (lstm2): LSTMCell(512, 512)
  (lstm3): LSTMCell(512, 512)
  (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_normalization2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_out): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=512, out_features=256, bias=True)
  (out_layer): Linear(in_features=256, out_features=82, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [15]:
batch_size = 5

In [26]:
import matplotlib.pyplot as plt
from IPython import display
from tqdm import tqdm
%matplotlib inline

criteron = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(lstm_cell_model.parameters(),lr=0.01)

def train(batch_size,epochs):
    losses=[]
    lstm_cell_model.train()
    h_i = lstm_cell_model.init_hidden(100,512,batch_size=batch_size)
    for epoch in range(epochs):
        l=0
        for i in range(len(network_output)//batch_size):
            lstm_cell_model.zero_grad()    
#             s = n_i[i*batch_size:(i+1)*batch_size]
            for k in range(sequence_length):
                a=lstm_cell_model(input_sequence = n_i[i*batch_size:(i+1)*batch_size,k,0:],hidden=h_i)
        #         print("Predicted output is: ",a.argmax(dim=1))
        #         print("Actual output is: ",network_output[i])
            loss = criteron(a,torch.tensor(network_output[i*batch_size:(i+1)*batch_size],device=device))
#             print("Loss is: ",loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            l+=loss.item()
            
        losses.append(loss//(len(network_output)//batch_size))
        plt.clf()
        plt.plot(losses, 'b-')
        plt.xlabel('Epoch (%d)' % epoch)
        plt.ylabel('Loss')
        display.display(plt.gcf())
        display.clear_output(wait=True)    
        

In [ ]:
train(5,1)

# TRAINING IS TILL HERE

# Now creating the output

In [384]:
pitchnames = sorted(set(item for item in notes))
start = np.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

h_1 = lstm_cell_model.init_hidden(100,512,1)
h_i = [i.to(device) for i in h_1]

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern)))
    prediction_input = prediction_input / float(n_vocab)
    
    for i in range(sequence_length):
        prediction = lstm_cell_model(torch.tensor(prediction_input[:,i:i+1]).float().to(device),h_i)

    index = np.argmax(prediction[-1].cpu().detach())
    result = int_to_note[index.item()]
    prediction_output.append(result)

    pattern=np.append(pattern,index)
    pattern = pattern[1:len(pattern)]

In [386]:
from music21 import stream
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Violin()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Violin()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.insert(instrument.Violin())

midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

# Till Here

In [310]:
a=lstm_cell_model(input_sequence = s[:,k,0:],hidden=h_i)

In [311]:
a.argmax(dim=1)

tensor([452, 450, 428, 510, 165])

In [312]:
network_output[i*batch_size:(i+1)*batch_size]

[199, 101, 445, 229, 292]

In [313]:
n_i[0:1,1].shape

torch.Size([1, 1])

In [314]:
network_output[0:1]

[241]

In [321]:
h_i = lstm_cell_model.init_hidden(100,512,2)
a=lstm_cell_model(input_sequence = n_i[1:3,1],hidden=h_i)

In [322]:
a.argmax(1)

tensor([510, 450])

# LSTM SIMPLE

In [995]:
class LSTM_simple(nn.Module):

    def __init__(self, hidden_size, sequence_length, n_vocab):
        super(LSTM_simple, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=3, batch_first=True,dropout = 0.3).float()        
#         self.lstm=nn.LSTM(1,hidden_size,dropout = 0.3,batch_first=False).float()
#         self.lstm2=nn.LSTM(hidden_size,hidden_size, dropout = 0.3,batch_first=False).float()
#         self.lstm3=nn.LSTM(hidden_size,hidden_size).float()
        self.batch_normalization = nn.BatchNorm1d(num_features=hidden_size)
        self.batch_normalization2 = nn.BatchNorm1d(num_features=256)
        self.drop_out = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_size,256)
        self.out_layer = nn.Linear(256,n_vocab)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
#         self.tanh = nn.Tanh()
        
        
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2output = nn.Linear(hidden_size, n_vocab)
        
    def init_hidden(self,sequence_length,hidden_size):
        return (torch.randn(3,1, hidden_size), torch.randn(3,1, hidden_size),
                torch.randn(1,1, hidden_size,dtype=torch.double), torch.randn(1,1, hidden_size,dtype=torch.double),
                torch.randn(1,1, hidden_size,dtype=torch.double), torch.randn(1,1, hidden_size,dtype=torch.double))

    def forward(self, input_sequence,hidden):
#         print(hidden[0].shape)
#         print(hidden[1].type())
#         print(input_sequence.shape)
        output,(h1,c1) = self.lstm(input_sequence,(hidden[0],hidden[1]))
#         output,(h2,c2) = self.lstm2(output,(hidden[2],hidden[3]))
#         output,(h3,c3) = self.lstm3(output,(hidden[4],hidden[5]))
#         print(output.squeeze().shape)
#         print(h1[-1].shape)
#         print(self.tanh(h1[2:3]))
#         output = h1[-1]
        output = self.batch_normalization(output.squeeze())
        output = self.drop_out(output)
        output = self.linear(output)
        output = self.relu(output)
        output = self.batch_normalization2(output.squeeze())
        output = self.drop_out(output)
        output = self.out_layer(output)
#         print(self.softmax(output[-1]))
        output = self.softmax(output)
        return output

In [996]:
model = LSTM_simple(512,100,n_vocab)

In [938]:
h_i = model.init_hidden(100,512)
a=model(input_sequence = n_i[0].unsqueeze(0).float(),hidden=h_i)

In [939]:
a.shape

torch.Size([1, 519])

In [940]:
n_vocab

519

In [941]:
network_output[0].shape

(519,)

In [942]:
# gather(1,acts.unsqueeze(1).to(device))
a.gather(0,torch.tensor([99,102]).unsqueeze(0))

RuntimeError: Expected tensor [1, 2], src [1, 519] and index [1, 2] to have the same size apart from dimension 0

In [943]:
torch.tensor([99,102]).unsqueeze(0)

tensor([[ 99, 102]])

In [944]:
a.shape

torch.Size([1, 519])

In [945]:
criteron = nn.CrossEntropyLoss()

In [997]:
optimizer = torch.optim.RMSprop(model.parameters(),lr=0.0001)
optimizer = torch.optim.Adam(model.parameters())

In [947]:
loss = criteron(a[-1].unsqueeze(0),torch.tensor([network_output[0].argmax()]))

In [948]:
loss

tensor(6.2519, grad_fn=<NllLossBackward>)

In [949]:
optimizer.zero_grad()

In [950]:
loss.backward()

In [951]:
optimizer.step()

In [952]:
torch.tensor(network_output[0]).shape

torch.Size([519])

In [953]:
torch.tensor(network_output[0]).argmax()

tensor(241)

In [954]:
network_output[0].argmax().shape

()

In [955]:
a=model(input_sequence = n_i[0].unsqueeze(0).float(),hidden=h_i)

In [956]:
a=model(input_sequence = n_i[0].unsqueeze(0).float(),hidden=h_i)
a[-1].argmax()

tensor(241)

In [957]:
loss = criteron(a[-1].unsqueeze(0),torch.tensor([481]))

In [958]:
loss

tensor(6.2517, grad_fn=<NllLossBackward>)

In [959]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [960]:
model.zero_grad()

In [961]:
model.zero_grad()

In [962]:
loss = criteron(a[-1].unsqueeze(0),torch.tensor([network_output[0].argmax()]))

In [963]:
a.shape

torch.Size([1, 519])

In [1023]:
model.train()
for i in range(20):
    model.zero_grad()
    a=model(input_sequence = n_i[i].unsqueeze(0).float(),hidden=h_i)
    print("Predicted output is: ",a.argmax().item())
    print("Actual output is: ",network_output[i].argmax())
    loss = criteron(a[-2:-1],torch.tensor([network_output[i].argmax()]))
    print("Loss is: ",loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Predicted output is:  404
Actual output is:  241
Loss is:  6.252182960510254
Predicted output is:  268
Actual output is:  451
Loss is:  6.248105049133301
Predicted output is:  332
Actual output is:  418
Loss is:  6.251610279083252
Predicted output is:  2056
Actual output is:  127
Loss is:  6.252231597900391
Predicted output is:  653
Actual output is:  378
Loss is:  6.251732349395752
Predicted output is:  194
Actual output is:  132
Loss is:  6.252285957336426
Predicted output is:  191
Actual output is:  378
Loss is:  6.251818656921387
Predicted output is:  1229
Actual output is:  132
Loss is:  6.251744747161865
Predicted output is:  170
Actual output is:  241
Loss is:  6.250189781188965
Predicted output is:  170
Actual output is:  208
Loss is:  6.252152919769287
Predicted output is:  433
Actual output is:  451
Loss is:  6.248319625854492
Predicted output is:  332
Actual output is:  127
Loss is:  6.249650955200195
Predicted output is:  170
Actual output is:  396
Loss is:  6.2511072158813

In [1077]:
model.train()
for j in range(20):
    i=7777
    model.zero_grad()
    a=model(input_sequence = n_i[i].unsqueeze(0).float()*n_vocab,hidden=h_i)
    print("Predicted output is: ",a[-1].argmax().item())
    print("Actual output is: ",network_output[i].argmax())
    loss = criteron(a[-2:-1],torch.tensor([network_output[i].argmax()]))
    print("Loss is: ",loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Predicted output is:  241
Actual output is:  502
Loss is:  6.252744674682617
Predicted output is:  241
Actual output is:  502
Loss is:  6.255208969116211
Predicted output is:  241
Actual output is:  502
Loss is:  6.250744342803955
Predicted output is:  241
Actual output is:  502
Loss is:  6.25487756729126
Predicted output is:  241
Actual output is:  502
Loss is:  6.255208969116211
Predicted output is:  241
Actual output is:  502
Loss is:  6.2551679611206055
Predicted output is:  451
Actual output is:  502
Loss is:  6.255159854888916
Predicted output is:  241
Actual output is:  502
Loss is:  6.2550368309021
Predicted output is:  241
Actual output is:  502
Loss is:  6.254593372344971
Predicted output is:  127
Actual output is:  502
Loss is:  6.255098342895508
Predicted output is:  241
Actual output is:  502
Loss is:  6.254260063171387
Predicted output is:  241
Actual output is:  502
Loss is:  6.25430965423584
Predicted output is:  241
Actual output is:  502
Loss is:  6.254878044128418
Pr

In [917]:
model.eval()
a=model(input_sequence = n_i[i].unsqueeze(0).float(),hidden=h_i)
print(a.argmax())

tensor(313)


In [918]:
network_output.shape

(67552, 519)

In [919]:
a=model(input_sequence = n_i[i].unsqueeze(0).float(),hidden=h_i)


In [920]:
a.max()

tensor(0.0024, grad_fn=<MaxBackward1>)

In [921]:
a.requires_grad_

<function Tensor.requires_grad_>

In [922]:
    loss = criteron(a,torch.tensor([network_output[0]])).float()


In [923]:
loss

tensor(0.0019, grad_fn=<MseLossBackward>)

In [924]:
loss.backward()

RuntimeError: expected dtype Float but got dtype Byte

In [976]:
c=model(input_sequence = n_i[7777].unsqueeze(0).float(),hidden=h_i)

In [977]:
c.argmax()

tensor(241)

In [978]:
model.eval()

LSTM_simple(
  (lstm): LSTM(1, 512, num_layers=3, batch_first=True, dropout=0.3)
  (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_normalization2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_out): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=512, out_features=256, bias=True)
  (out_layer): Linear(in_features=256, out_features=519, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=-1)
  (hidden2output): Linear(in_features=512, out_features=519, bias=True)
)

In [1021]:
c=model(input_sequence = n_i[11].unsqueeze(0).float(),hidden=h_i)

In [1014]:
loss = criteron(c[-2:-1],torch.tensor([network_output[11].argmax()]))

In [1015]:
loss

tensor(6.2520, grad_fn=<NllLossBackward>)

In [1016]:
torch.tensor([network_output[10].argmax()])

tensor([451])

In [1017]:
loss.backward()

In [1018]:
optimizer.step()

In [1019]:
c[-1].shape

torch.Size([519])

In [1022]:
c[-1].argmax()

tensor(258)

In [1070]:
a=torch.gather(c,0,torch.tensor([0]).unsqueeze(0))

RuntimeError: Expected tensor [1, 1], src [100, 519] and index [1, 1] to have the same size apart from dimension 0

In [1033]:
a.shape

torch.Size([100, 519])

In [1062]:
so = [[np]*519]*100

In [1063]:
so[0]

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [1064]:
so[99]=[0]*519

In [1065]:
torch.tensor(so)

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [1068]:
torch.tensor([0]).unsqueeze(0).shape

torch.Size([1, 1])

In [1053]:
so[99]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [1073]:
network_input[0]*n_vocab

array([[ 28.],
       [467.],
       [466.],
       [399.],
       [488.],
       [495.],
       [437.],
       [451.],
       [488.],
       [481.],
       [451.],
       [418.],
       [176.],
       [101.],
       [418.],
       [335.],
       [487.],
       [486.],
       [314.],
       [436.],
       [335.],
       [468.],
       [437.],
       [467.],
       [434.],
       [435.],
       [335.],
       [314.],
       [229.],
       [450.],
       [449.],
       [514.],
       [495.],
       [481.],
       [494.],
       [399.],
       [488.],
       [445.],
       [459.],
       [  6.],
       [437.],
       [176.],
       [418.],
       [418.],
       [451.],
       [335.],
       [487.],
       [486.],
       [314.],
       [436.],
       [515.],
       [451.],
       [452.],
       [434.],
       [435.],
       [399.],
       [467.],
       [335.],
       [451.],
       [314.],
       [436.],
       [208.],
       [515.],
       [101.],
       [192.],
       [460.],
       [12

In [1074]:
network_input[1]*n_vocab

array([[467.],
       [466.],
       [399.],
       [488.],
       [495.],
       [437.],
       [451.],
       [488.],
       [481.],
       [451.],
       [418.],
       [176.],
       [101.],
       [418.],
       [335.],
       [487.],
       [486.],
       [314.],
       [436.],
       [335.],
       [468.],
       [437.],
       [467.],
       [434.],
       [435.],
       [335.],
       [314.],
       [229.],
       [450.],
       [449.],
       [514.],
       [495.],
       [481.],
       [494.],
       [399.],
       [488.],
       [445.],
       [459.],
       [  6.],
       [437.],
       [176.],
       [418.],
       [418.],
       [451.],
       [335.],
       [487.],
       [486.],
       [314.],
       [436.],
       [515.],
       [451.],
       [452.],
       [434.],
       [435.],
       [399.],
       [467.],
       [335.],
       [451.],
       [314.],
       [436.],
       [208.],
       [515.],
       [101.],
       [192.],
       [460.],
       [127.],
       [29

In [1085]:
lstm_model = LSTM(512,100,n_vocab)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [1086]:
h_i = lstm_model.init_hidden(100,512)
a=lstm_model(input_sequence = n_i[i].unsqueeze(0).float(),hidden=h_i)

torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [1088]:
a.shape

torch.Size([100, 519])

In [1090]:
lstm_model.train()
for j in range(20):
    i=7777
    a=lstm_model(input_sequence = n_i[i].unsqueeze(0).float()*n_vocab,hidden=h_i)
    print("Predicted output is: ",a[-1].argmax().item())
    print("Actual output is: ",network_output[i].argmax())
    loss = criteron(a[-2:-1],torch.tensor([network_output[i].argmax()]))
    print("Loss is: ",loss.item())
    loss.backward()
    optimizer.step()

torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  324
Actual output is:  502
Loss is:  6.25221586227417


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  195
Actual output is:  502
Loss is:  6.2497758865356445
torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  385
Actual output is:  502
Loss is:  6.2518157958984375
torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  273
Actual output is:  502
Loss is:  6.25088357925415
torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  461
Actual output is:  502
Loss is:  6.253246784210205
torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  455
Actual output is:  502
Loss is:  6.2498908042907715
torch.Size([1, 100, 512])
torch.FloatTensor
torch.Size([1, 100, 1])
torch.Size([100, 512])
Predicted output is:  455
Actual output is:  502
Loss is:  6.249485

In [1118]:
pitchnames = sorted(set(item for item in notes))
start = np.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

h_1 = model.init_hidden(100,512)

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model(torch.tensor(prediction_input).float(),h_1)

    index = np.argmax(prediction[-1].detach())
    result = int_to_note[index.item()]
    prediction_output.append(result)

    pattern=np.append(pattern,index)
    pattern = pattern[1:len(pattern)]

In [1120]:
prediction_output

['1.4.7.10',
 '4.8.11',
 '2.5.8.10',
 '5.8.9',
 '2.3.6',
 '4.8.11',
 '7.9.11.1.3.4',
 '11.3',
 '10.1',
 '4.8.11',
 '4.8.11',
 '4.8.11',
 'B3',
 '5.10',
 '4.8.11',
 'E5',
 '4.8.11',
 '4.8.11',
 '1.7',
 '3.7.10',
 '4.8.11',
 '5.6',
 '10.0.3',
 '7.8.0',
 '8.9.1.2',
 '4.8.11',
 'F2',
 'E-4',
 '4.8.11',
 'E5',
 '7.9.11.1.3.4',
 '4.8.11',
 '0.3.6.8',
 '4.8.11',
 '0.1.6',
 '10.0.3',
 '11.4',
 'D3',
 '3.6.7',
 '10.1.3',
 '10.0.3',
 '4.8.11',
 '4.8.11',
 '3.6.8',
 '6.9.10.1',
 '4.8.11',
 'A0',
 'B3',
 '4.8.11',
 '4.8.11',
 '10.11.4',
 'C2',
 'E5',
 '3.5.9',
 'E5',
 '3.5.10',
 '3.4.5',
 '4.8.11',
 '11.2.4.7',
 '4.8.11',
 '2.5.6.9',
 'F#4',
 '3.5.6',
 '4.8.11',
 '9.11',
 '4.8.11',
 '4.8.11',
 '4.8.11',
 '4.8.11',
 '10.2.5',
 '7.9.0.2',
 '9.1.3',
 '3.7',
 '4.8.11',
 '7.9.11.1.3.4',
 '4.5.10',
 'E5',
 '4.8.11',
 '0.3.6.8',
 'E5',
 '4.8.11',
 '9.11',
 '4.8.11',
 'E5',
 '5.6.10.1',
 '4.8.11',
 '4.8.11',
 '7.11.0',
 '4.8.11',
 '4.8.11',
 '9.11.3.4',
 '4.8.11',
 '4.8.11',
 '4.8.11',
 '4.8.11',
 '4.7.8'

In [1123]:
from music21 import stream
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

In [1106]:
prediction.shape

torch.Size([100, 519])

In [1108]:
type(pattern)

numpy.ndarray

In [1111]:
type(network_input)

numpy.ndarray

In [1115]:
network_input.app

[[0.05394990366088632],
 [0.8998073217726397],
 [0.8978805394990366],
 [0.7687861271676301],
 [0.9402697495183044],
 [0.953757225433526],
 [0.8420038535645472],
 [0.8689788053949904],
 [0.9402697495183044],
 [0.9267822736030829],
 [0.8689788053949904],
 [0.8053949903660886],
 [0.33911368015414256],
 [0.19460500963391136],
 [0.8053949903660886],
 [0.6454720616570327],
 [0.9383429672447013],
 [0.9364161849710982],
 [0.605009633911368],
 [0.8400770712909441],
 [0.6454720616570327],
 [0.9017341040462428],
 [0.8420038535645472],
 [0.8998073217726397],
 [0.8362235067437379],
 [0.838150289017341],
 [0.6454720616570327],
 [0.605009633911368],
 [0.441233140655106],
 [0.8670520231213873],
 [0.8651252408477842],
 [0.9903660886319846],
 [0.953757225433526],
 [0.9267822736030829],
 [0.9518304431599229],
 [0.7687861271676301],
 [0.9402697495183044],
 [0.8574181117533719],
 [0.884393063583815],
 [0.011560693641618497],
 [0.8420038535645472],
 [0.33911368015414256],
 [0.8053949903660886],
 [0.80539499